In [25]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [26]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [31]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [32]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [33]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9], device='cuda:0')


In [34]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [35]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [36]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [37]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1794, -0.2547,  0.1062, -0.1531, -0.0845,  0.4633, -0.1216, -0.2452,
          0.4205,  0.0996, -0.5180,  0.0223, -0.0606,  0.0164,  0.2026, -0.3941,
          0.4969,  0.0662, -0.8144,  0.2485],
        [-0.1561,  0.1757,  0.3723, -0.1792,  0.0370,  0.3199,  0.1485, -0.3024,
          0.0397, -0.2854, -0.4161, -0.0143, -0.0195,  0.5180, -0.0544, -0.4949,
          0.1786,  0.2946, -0.0053,  0.1990],
        [-0.1388,  0.1523,  0.5057, -0.5176, -0.1517,  0.2423, -0.1126, -0.5426,
          0.2444,  0.0607, -0.1756, -0.0166, -0.4576,  0.2272,  0.1346, -0.3238,
         -0.0306,  0.1024, -0.3974,  0.4990]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1794, 0.0000, 0.1062, 0.0000, 0.0000, 0.4633, 0.0000, 0.0000, 0.4205,
         0.0996, 0.0000, 0.0223, 0.0000, 0.0164, 0.2026, 0.0000, 0.4969, 0.0662,
         0.0000, 0.2485],
        [0.0000, 0.1757, 0.3723, 0.0000, 0.0370, 0.3199, 0.1485, 0.0000, 0.0397,
         0.0000, 0.0000, 0.0000, 0.0000, 0.5180, 0.00

In [40]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)
print(logits)

tensor([[-0.1220, -0.0103,  0.2287,  0.1476, -0.0803, -0.0783, -0.0489, -0.1088,
         -0.1602,  0.1245],
        [-0.2078,  0.0757,  0.3534,  0.1411, -0.1548, -0.0083, -0.1272, -0.0457,
         -0.0657,  0.0647],
        [-0.1530, -0.1209,  0.1445,  0.2684, -0.1839,  0.0151,  0.0111, -0.0652,
         -0.2121,  0.0272]], grad_fn=<AddmmBackward0>)


In [47]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.0888, 0.0993, 0.1261, 0.1162, 0.0925, 0.0927, 0.0955, 0.0899, 0.0854,
         0.1136],
        [0.0800, 0.1062, 0.1402, 0.1134, 0.0844, 0.0977, 0.0867, 0.0941, 0.0922,
         0.1051],
        [0.0872, 0.0901, 0.1174, 0.1329, 0.0846, 0.1032, 0.1028, 0.0952, 0.0822,
         0.1044]], grad_fn=<SoftmaxBackward0>)


In [49]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0231,  0.0257,  0.0225,  ..., -0.0247, -0.0007, -0.0216],
        [ 0.0278, -0.0134, -0.0277,  ...,  0.0012, -0.0197,  0.0303]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0203, -0.0124], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0258, -0.0173,  0.0262,  ..., -0.0203,  0.0382, -0.0177],
        [ 0.0270, -0.0071, -0.0312,  ..., -0.0419, -0.0227,  0.0378]],
       device='cuda:0', grad_fn=<Sl